In [ ]:
#| hide
import os
import joblib
from execnb.nbio import read_nb, write_nb, new_nb, mk_cell
from nbdev.process import NBProcessor, Processor

# Syncing through nbdev

> Showcasing the syncing capability

In [ ]:
%load_ext nbmodular.core.cell2func

## simple case: each function is written in a single cell

This is markdown

In [ ]:
%%function first_function
a = 3 
print ('a', a)

a 3
Stored the following local variables in the first_function current_values dictionary: ['a']


In [ ]:
# normal code cell
print ('outside', a)

outside 3


In [ ]:
%%function second_function
b = 4
c = a+b
print (a, b, c)

3 4 7
Stored the following local variables in the second_function current_values dictionary: ['b', 'c']


In [ ]:
cell_processor = %cell_processor

In [ ]:
[function['name'] for function in cell_processor.cell_nodes]

['first_function', 'second_function']

In [ ]:
print ('last cell:', a, b, c)

last cell: 3 4 7


In [ ]:
class FunctionProcessor(Processor):
    def __init__ (self, function_cells):
        self.function_cells = function_cells
        self.function_idx = 0
    
    def cell(self, cell):
        if '%%function' in cell.source:
            print ('\n**********************************************************************')
            print (self.function_cells[self.function_idx]['code'])
            self.function_idx += 1
            print ('**********************************************************************\n')
        else:
            print (cell.source)

            
function_processor = FunctionProcessor (cell_processor.cell_nodes)
NBProcessor ('./nbdev_sync.ipynb', function_processor).process()

import os
import joblib
from execnb.nbio import read_nb, write_nb, new_nb, mk_cell
from nbdev.process import NBProcessor, Processor
# Syncing through nbdev

> Showcasing the syncing capability
%load_ext nbmodular.core.cell2func
## simple case: each function is written in a single cell
This is markdown

**********************************************************************
def first_function():
    a = 3 
    print ('a', a)
    return a

**********************************************************************

# normal code cell
print ('outside', a)

**********************************************************************
def second_function():
    b = 4
    c = a+b
    print (a, b, c)
    return c,b

**********************************************************************

cell_processor = %cell_processor
[function['name'] for function in cell_processor.cell_nodes]
print ('last cell:', a, b, c)

**********************************************************************


IndexError: list index out of range

In [ ]:
function_processor.cell_nodes

AttributeError: 'FunctionProcessor' object has no attribute 'cell_nodes'

## second case: functions are split into different cells

In [ ]:
%%function myf
print ('hello')
a = 3

hello
Stored the following local variables in the myf current_values dictionary: ['a']


In [ ]:
%print myf

def myf():
    print ('hello')
    a = 3
    return a



In [ ]:
myf_info.original_cell

"print ('hello')\na = 3\n"

In [ ]:
%%function myf --merge
b = 4

Stored the following local variables in the myf current_values dictionary: ['b']
hello
Stored the following local variables in the myf current_values dictionary: ['a', 'b']


In [ ]:
%print myf

def myf():
    print ('hello')
    a = 3
    b = 4
    return b,a



In [ ]:
%%function myf --merge
c = 5
d = a+b+c

Stored the following local variables in the myf current_values dictionary: ['c', 'd']
hello
Stored the following local variables in the myf current_values dictionary: ['a', 'b', 'c', 'd']


In [ ]:
%print myf

def myf():
    print ('hello')
    a = 3
    b = 4
    c = 5
    d = a+b+c
    return c,b,d,a



In [ ]:
myf_info.original_cell

"print ('hello')\na = 3\nb = 4\nc = 5\nd = a+b+c\n"

In [ ]:
from sklearn.utils import Bunch

In [ ]:
self = myf_info
self.cell_nodes = []

In [ ]:


# TODO in cell_processor:

self.cell_nodes_per_function[this_function.name] += this_function.copy() # alternative: only code_parts are stored
# merge code: determine first, unique, last